In [1]:
# import usual suspects
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy.ndimage

In [2]:
model_path = "model/Kaggle-MNIST-Multilayer-Perceptron-1.ckpt"

In [3]:
# load the mnist training data CSV file into a list
# training_data_file = open("mnist_train_100.csv", 'r')
training_data_file = open("train.csv", 'r')
training_data_list = training_data_file.readlines()
print ("length of Original Training file", len(training_data_list))

# Take out first row as it contain columns name

# create a validtion data list, first 20 % rows
# validation_data_list = training_data_list[1:8401]

validation_data_list = training_data_list[1:int(len(training_data_list) * .20)]
print ("length of Validation file", len(validation_data_list))

# create a final training data list remaining 80 rows
# training_data_list = training_data_list[8401:]
training_data_list = training_data_list[int(len(training_data_list) * .20):]

print ("length of Training file", len(training_data_list))

training_data_file.close()

length of Original Training file 42001
length of Validation file 8399
length of Training file 33601


In [4]:
trainX = []
trainY = []

# now go to each record in batch of training data list
for record in training_data_list:
    # split the record by the ',' commas
    all_rows = record.split(',')

    # input should only have pixel so take out label column out of it
    # which is first column in data so [1:]
#     image = np.asfarray(all_rows[1:])
    image = (np.asfarray(all_rows[1:]) / 256.0 * 0.99) + 0.01
    trainX.append(image)
                
    # label is first column
    label = np.zeros([10]) + 0.01
    label[int(all_rows[0])] = 0.99
    trainY.append(label)

print(len(trainX))
print(len(trainY))


33601
33601


In [5]:
validationX = []
validationY = []

# now go to each record in batch of training data list
for record in validation_data_list:
    # split the record by the ',' commas
    all_rows = record.split(',')

    # input should only have pixel so take out label column out of it
    # which is first column in data so [1:]
#     image = (np.asfarray(all_rows[1:]) / 256.0 * 0.99) + 0.01
    image = (np.asfarray(all_rows[1:]) / 256.0 * 0.99) + 0.01
    validationX.append(image)
                
    # label is first column
    label = np.zeros([10]) + 0.01
    label[int(all_rows[0])] = 0.99
    validationY.append(label)

print(len(validationX))
print(len(validationY))

8399
8399


In [6]:
trainX_plusx_img = []
trainY_plusx_img = []

# now go to each record in batch of training data list
for record in training_data_list:
    # split the record by the ',' commas
    all_rows = record.split(',')

    # input should only have pixel so take out label column out of it
    # which is first column in data so [1:]
#     image = np.asfarray(all_rows[1:])
    image = (np.asfarray(all_rows[1:]) / 256.0 * 0.99) + 0.01
    
    # create new training data by rotating anticlockwise by 10 degrees
    image = scipy.ndimage.interpolation.rotate(image.reshape(28,28), angle= 18, cval=0.01, order=1, reshape=False)
    image = image.reshape(784)
    trainX_plusx_img.append(image)
                
    # label is first column
    label = np.zeros([10]) + 0.01
    label[int(all_rows[0])] = 0.99
    trainY_plusx_img.append(label)

print(len(trainX_plusx_img))
print(len(trainY_plusx_img))

33601
33601


In [7]:
trainX_minusx_img = []
trainY_minusx_img = []

# now go to each record in batch of training data list
for record in training_data_list:
    # split the record by the ',' commas
    all_rows = record.split(',')

    # input should only have pixel so take out label column out of it
    # which is first column in data so [1:]
#     image = np.asfarray(all_rows[1:])
    image = (np.asfarray(all_rows[1:]) / 256.0 * 0.99) + 0.01
    # create new training data by rotating clockwise by 10 degrees
    image = scipy.ndimage.interpolation.rotate(image.reshape(28,28), angle= -18, cval=0.01, order=1, reshape=False)
    image = image.reshape(784)
    trainX_minusx_img.append(image)
                
    # label is first column
    label = np.zeros([10]) + 0.01
    label[int(all_rows[0])] = 0.99
    trainY_minusx_img.append(label)

print(len(trainX_minusx_img))
print(len(trainY_minusx_img))

33601
33601


In [8]:
#  Add three above dataset into one

final_trainX = trainX + trainX_plusx_img + trainY_minusx_img
final_trainY = trainY + trainY_plusx_img + trainY_minusx_img

print (len(final_trainX))
print (len(final_trainY))

100803
100803


In [9]:
def next_batch(data, batch_size, batch_number):
    start = 0
    stop = 0
    if batch_number == 0:
        return data[start:batch_size]
    else:
        for b in range(batch_number):
            start = (b+1)*batch_size
            stop = (b+2)*batch_size
        
        return data[int(start):int(stop)]

In [10]:
# setup hyper parameters (learning rate, training epochs, batch size, display step)
learning_rate = 0.001
training_epochs = 3
batch_size = 256
display_step = 1

In [11]:
# setup network parameters (size of different hidden layers, size of input features, size of output classes)
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [12]:
# setup input placeholders for X with shape of (None initial, n_input ) and Y with shape (None initial , n_classes )
X = tf.placeholder(tf.float32, shape=[None, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_classes])


In [13]:
# setup weight and bias variables
W = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='h1'),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='h2'),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='w_out')
}

b = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='b_out')
}

In [14]:
# create a function name multilayer_perceptron(x,weights, biases) and return out_layer

def multilayer_perceptron(x, weights, bias):
    # 1 hidden layer with Linear and then RELU Activation
    layer1 = tf.add(tf.matmul(x, weights['h1']), bias['b1'])
    layer1 = tf.nn.relu(layer1)
    
    # 2 hidden layer with Linear and then RELU Activation
    layer2 = tf.add(tf.matmul(layer1, weights['h2']), bias['b2'])
    layer2 = tf.nn.relu(layer2)
    
    # Outut layer with Linear Activation
    out_layer = tf.add(tf.matmul(layer2, weights['out']), bias['out'])
    
    return out_layer



In [15]:
# setup pred model
pred = multilayer_perceptron(X, W, b)

# setup cost which is average of cross entropy function aka -ylog(pred)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))

# setup optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [16]:
# initialzie all variables
init = tf.global_variables_initializer()

In [17]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [18]:
# run the graph within session

with tf.Session() as sess:
    sess.run(init)
    
    # loop through all the range of epochs
    for epoch in range(training_epochs):
        avg_cost = 0
        
        #get total batches and loop through range of them
        total_batches = int(len(training_data_list) / batch_size)
        for batch in range(total_batches):
            
            # Train & run optimzer, cost in same session and feed values to Variables
            o, c = sess.run([optimizer, cost], feed_dict={X: next_batch(trainX, batch_size, batch), Y: next_batch(trainY, batch_size, batch)})
            
            #compute average cost
            avg_cost += c / total_batches
#             if (epoch % display_step) == 0:
#                 print ('batch=', batch+1, 'Cost=', c, 'Average Cost=', avg_cost, ' for Total batches', total_batches)
        
        # Test Our Trained Model for every epoch
        correct_predection = tf.equal(tf.argmax(pred, axis=1), tf.argmax(Y, axis=1))
        
        # Calculate Accuracy aftering testing
        accuracy = tf.reduce_mean(tf.cast(correct_predection, tf.float32))
            
        # append correct or incorrect to list
        accuracy_eval = accuracy.eval({X: validationX, Y: validationY})
        
        # print display step with epoch, and cost
        if (epoch % display_step) == 0:
            print ('Epoch=', epoch+1, 'Avg Cost=', avg_cost, 'Accuracy=', accuracy_eval)
    
    # print optimization finished for all epochs
    print ("Optimization Finished for all Epochs!")
    
    # Save model weights to local disk
    save_path = saver.save(sess, model_path)
    print ("Current Pred Model saved in file: %s" % save_path)

Epoch= 1 Avg Cost= 468.425053254 Accuracy= 0.718776
Epoch= 2 Avg Cost= 204.053129007 Accuracy= 0.781998
Epoch= 3 Avg Cost= 192.438835785 Accuracy= 0.803905
Optimization Finished for all Epochs!
Current Pred Model saved in file: model/Kaggle-MNIST-Multilayer-Perceptron-1.ckpt


In [19]:
# Now, to test our Model on Kaggle Test data, 

In [20]:
# let's load the mnist test data CSV file into a list
test_data_file = open("test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_list = test_data_list[1:]
test_data_file.close()

print ("len of kaggle test data without column name", len(test_data_list) )

len of kaggle test data without column name 28000


In [21]:
testX = []
# for saving test file output
test_file_output = []
ImageIdIndex = 0

# now go to each record in batch of training data list
for record in test_data_list:
    
    # append empty list for every record
    test_file_output.append([])

    # add ImageId to empyt list
    test_file_output[ImageIdIndex].append(ImageIdIndex+1)

    # split the record by the ',' commas
    all_rows = record.split(',')

    # input should only have pixel so take out label column out of it
    # which is first column in data so [1:]
#     image = np.asfarray(all_rows[1:])
    image = (np.asfarray(all_rows[:]) / 256.0 * 0.99) + 0.01
    testX.append(image)

print(len(testX))


28000


In [22]:

with tf.Session() as sess:
    sess.run(init)
    
    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path)
    print ("Model restored from file:", save_path)
        
    # Now, query kaggle test data with our trained Neural Network
    predictions = sess.run(pred, feed_dict={X: testX})

    # the index of the highest value corresponds to the label
    predictions = [np.argmax(p) for p in predictions]
    
    result = pd.DataFrame({'ImageId': range(1,len(predictions)+1), 'Label': predictions})
    result.to_csv('output18.csv', index=False, encoding='utf-8')
        
print ("Submission File is Created, use latest output18.csv file to upload to kaggle, to see results")

Model restored from file: model/Kaggle-MNIST-Multilayer-Perceptron-1.ckpt
Submission File is Created, use latest output18.csv file to upload to kaggle, to see results
